In [9]:
import numpy as np
from json import load
import spacy
import nltk
import string
from nltk.corpus import stopwords
from string import punctuation
from spacy.matcher import PhraseMatcher
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, RocCurveDisplay, auc, f1_score, recall_score, precision_score
import joblib
nlp = spacy.load("ru_core_news_sm")


In [10]:
with open('../../Data/Datasets/data.json', 'r', encoding='utf-8') as js:
    discription_of_event = load(js)


In [11]:
kcm = {'знать': [], 'уметь': [], 'владеть': []}
for i in discription_of_event.values():
    for j in i:
        for z in i[j]:
            kcm[j].append(list(nlp(z).vector))


In [12]:
len(kcm.values()), len(kcm['знать']), len(kcm['знать'][0])


(3, 228, 96)

In [13]:
k = []
for i in kcm:
    for j in kcm[i]:
        k.append(j+[i])
# pd.DataFrame(k, columns=range(len(k[0])))
# sorted(k, key=len, reverse=False)
# k

In [14]:
len(k), len(k[0])

(707, 97)

In [15]:
data = pd.DataFrame(k, columns=range(len(k[0])))

In [16]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(
    96, axis=1), data[96], test_size=0.25, random_state=42)


# Bayes

# Подбираем гиппер-параметры.

In [17]:
gaussian_naive_bayes_optimal_f1 = GridSearchCV(GaussianNB(),
                                                        {'var_smoothing': np.arange(0, 1, 1e-1),
                                                         },
                                                        scoring=f1_score).fit(X_train, y_train)
gaussian_naive_bayes_optimal_precission = GridSearchCV(GaussianNB(),
                                                        {'var_smoothing': np.arange(0, 1, 1e-1),
                                                         },
                                                        scoring=f1_score).fit(X_train, y_train)
gaussian_naive_bayes_optimal_recall = GridSearchCV(GaussianNB(),
                                                        {'var_smoothing': np.arange(0, 1, 1e-1),
                                                         },
                                                        scoring=f1_score).fit(X_train, y_train)


c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: f1_score() takes 2 positional arguments but 3 were given

  warnings.warn(
c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Mike\Desktop\ITS_project\ITS_project\lib\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: f1_score() takes 2 

# Формула Байеса

In [18]:
y_predicted_by_gaussian_naive_bayes_f1 = gaussian_naive_bayes_optimal_f1.predict(X_test)
y_predicted_by_gaussian_naive_bayes_precission = gaussian_naive_bayes_optimal_precission.predict(X_test)
y_predicted_by_gaussian_naive_bayes_recall = gaussian_naive_bayes_optimal_recall.predict(X_test)

# Оценка качества 

In [19]:
print('Accuracy')
print(accuracy_score(y_test, y_predicted_by_gaussian_naive_bayes_f1))
print('Confusion matrix')
print(confusion_matrix(y_test, y_predicted_by_gaussian_naive_bayes_f1))
print('Precision, Recall, F\n', classification_report(
    y_test, y_predicted_by_gaussian_naive_bayes_f1))

Accuracy
0.903954802259887
Confusion matrix
[[39  2  2]
 [ 7 55  3]
 [ 3  0 66]]
Precision, Recall, F
               precision    recall  f1-score   support

     владеть       0.80      0.91      0.85        43
       знать       0.96      0.85      0.90        65
       уметь       0.93      0.96      0.94        69

    accuracy                           0.90       177
   macro avg       0.90      0.90      0.90       177
weighted avg       0.91      0.90      0.90       177



In [20]:
print('Accuracy')
print(accuracy_score(y_test, y_predicted_by_gaussian_naive_bayes_recall))
print('Confusion matrix')
print(confusion_matrix(y_test, y_predicted_by_gaussian_naive_bayes_recall))
print('Precision, Recall, F\n', classification_report(
    y_test, y_predicted_by_gaussian_naive_bayes_recall))



Accuracy
0.903954802259887
Confusion matrix
[[39  2  2]
 [ 7 55  3]
 [ 3  0 66]]
Precision, Recall, F
               precision    recall  f1-score   support

     владеть       0.80      0.91      0.85        43
       знать       0.96      0.85      0.90        65
       уметь       0.93      0.96      0.94        69

    accuracy                           0.90       177
   macro avg       0.90      0.90      0.90       177
weighted avg       0.91      0.90      0.90       177



In [21]:
print('Accuracy')
print(accuracy_score(y_test, y_predicted_by_gaussian_naive_bayes_precission))
print('Confusion matrix')
print(confusion_matrix(y_test, y_predicted_by_gaussian_naive_bayes_precission))
print('Precision, Recall, F\n', classification_report(
    y_test, y_predicted_by_gaussian_naive_bayes_precission))


Accuracy
0.903954802259887
Confusion matrix
[[39  2  2]
 [ 7 55  3]
 [ 3  0 66]]
Precision, Recall, F
               precision    recall  f1-score   support

     владеть       0.80      0.91      0.85        43
       знать       0.96      0.85      0.90        65
       уметь       0.93      0.96      0.94        69

    accuracy                           0.90       177
   macro avg       0.90      0.90      0.90       177
weighted avg       0.91      0.90      0.90       177



In [22]:
# joblib.dump(gaussian_naive_bayes,
#             '../../PipeLines/Classifications/NaiveBayesKCMDiscriptor/nb_classification_discriptor_kcm.sav')
